In [2]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.1, min_tracking_confidence=0.1)

def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

object_position = [300, 300] 
object_radius = 30           
is_dragging = False          

cap = cv2.VideoCapture(0)

while cap.isOpened(): 
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    frame_height, frame_width, _ = frame.shape

    cv2.circle(frame, (object_position[0], object_position[1]), object_radius, (255, 0, 0), -1)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            landmarks = hand_landmarks.landmark
            thumb_tip = (int(landmarks[mp_hands.HandLandmark.THUMB_TIP].x * frame_width),
                         int(landmarks[mp_hands.HandLandmark.THUMB_TIP].y * frame_height))
            index_tip = (int(landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * frame_width),
                         int(landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * frame_height))

            pinch_distance = calculate_distance(thumb_tip, index_tip)

            if pinch_distance < 40: 
                cv2.putText(frame, "Pinch Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                if not is_dragging:
                    distance_to_object = calculate_distance(thumb_tip, object_position)
                    if distance_to_object < object_radius:
                        is_dragging = True

                if is_dragging:
                    object_position[0], object_position[1] = thumb_tip

            else:
                is_dragging = False
    cv2.imshow("Drag and Drop", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()